<a href="https://colab.research.google.com/github/jinseriouspark/polaris-llm/blob/main/gemma_finetuning_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction Fine-tuning

- 참고자료 : https://github.com/tsdata/langchain-ollama/blob/main/006_gemma_peft_qlora_colab/gemma_finetuning_koalpaca.ipynb

In [1]:
# 데이터셋 다운로드
!pip install -U datasets==2.17.0

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [3]:
from datasets import load_dataset

dataset = load_dataset("royboy0416/ko-alpaca")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 49620
    })
})


In [4]:
dataset['train'][3]

{'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
 'input': '',
 'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n대기 오염을 어떻게 줄일 수 있나요?\n\n### Response:\n대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'}

## Gemma dataset formatting

- 참고자료 https://ai.google.dev/gemma/docs/formatting?hl=ko

```<start_of_turn>user
knock knock<end_of_turn>
<start_of_turn>model
who is there<end_of_turn>
<start_of_turn>user
Gemma<end_of_turn>
<start_of_turn>model
Gemma who?<end_of_turn>
```

In [5]:
def gemma_formatting(example):

  # if 'input' has data:
  if example['input'] and len(example['input']) > 0:
    text = f'''<start_of_turn>user\n{example['instruction']}\n{example['input']}<end_of_turn>\n<start_of_turn>model\n{example['output']}\n<end_of_turn>'''
  else:
    text = f'''<start_of_turn>user\n{example['instruction']}\n<end_of_turn>\n<start_of_turn>model\n{example['output']}\n<end_of_turn>'''
  return {'prompt': text}

# dataset update

dataset = dataset.map(gemma_formatting)

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [6]:
dataset['train'][3]

{'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
 'input': '',
 'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n대기 오염을 어떻게 줄일 수 있나요?\n\n### Response:\n대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.',
 'prompt': '<start_of_turn>user\n대기 오염을 어떻게 줄일 수 있나요?\n<end_of_turn>\n<start_of_turn>model\n대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.\n<end_of_turn>'}

## model load and tuning

1. model loading : gemma-7b load & instruction fine-tuning
2. evaluation : ??

In [7]:
!pip install -qU transformers==4.38.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10

In [8]:
!nvidia-smi

Fri May 24 07:01:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   52C    P8              17W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login


In [10]:
notebook_login()

In [11]:
 model_id = 'google/gemma-7b-it'

In [12]:
bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                 bnb_4bit_quant_type = 'nf4',
                                 bnb_4bit_compute_dtype = torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map = {"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
tokenizer.pad_token = tokenizer.eos_token

In [15]:
dataset = dataset.map(lambda samples: tokenizer(samples['prompt']),
                      batched = True)
dataset = dataset['train'].train_test_split(test_size = 0.2)
dataset

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 39696
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 9924
    })
})

In [16]:
train_data = dataset['train']
test_data = dataset['test']

In [17]:
print(train_data[0])

{'instruction': 'AI가 환경에 미치는 부정적인 영향 두 가지를 추가하십시오.', 'input': '', 'output': 'AI가 지능을 향상하고 인간 대신 일하는 능력을 증가시키는 것 외에도 프라이버시 문제와 더불어 일부 작업 분야에 대한 잠재적인 업무 손실을 가져올 수 있습니다.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\nAI가 환경에 미치는 부정적인 영향 두 가지를 추가하십시오.\n\n### Response:\nAI가 지능을 향상하고 인간 대신 일하는 능력을 증가시키는 것 외에도 프라이버시 문제와 더불어 일부 작업 분야에 대한 잠재적인 업무 손실을 가져올 수 있습니다.', 'prompt': '<start_of_turn>user\nAI가 환경에 미치는 부정적인 영향 두 가지를 추가하십시오.\n<end_of_turn>\n<start_of_turn>model\nAI가 지능을 향상하고 인간 대신 일하는 능력을 증가시키는 것 외에도 프라이버시 문제와 더불어 일부 작업 분야에 대한 잠재적인 업무 손실을 가져올 수 있습니다.\n<end_of_turn>', 'input_ids': [2, 106, 1645, 108, 11716, 236361, 161409, 237392, 236179, 48835, 237924, 236214, 43761, 236864, 85024, 52604, 240112, 93828, 128753, 236791, 133818, 197538, 235265, 108, 107, 108, 106, 2516, 108, 11716, 236361, 34805, 239474, 236392, 220234, 237047, 48060, 30743, 238264, 26801, 237502, 32929, 40284, 235248, 239474, 238365, 

In [33]:
def get_completion(query: str, model, tokenizer):
  prompt_template = f"""<start_of_turn>user
                      {query}<end_of_turn>
                      <start_of_turn>
                      model
                      """
  prompt = prompt_template.format(query = query)
  encoded = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encoded.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_token = True)
  return decoded

In [34]:
# base model response
result = get_completion("회사 때려치고 싶어. 그만둘까?", model, tokenizer)
print(result)

<bos><start_of_turn>user
                      회사 때려치고 싶어. 그만둘까?<end_of_turn>
                      <start_of_turn>
                      model
                      <eos>은 회사를 떠나거나 그만둘 것에 대한 고려사입니다. 회사를 떠날 수 있는 많은 요인이 있지만, 그만둘 것만은 아닙니다.

**회사를 떠날 수 있는 요인:**

* **업무 불만족:** 회사의 업무에 불만족하거나 업무를 잘 못하는 것 같다.
* **사무 부담:** 회사의 사무 부담이 너무 많거나 불안한 것 같다.
* **인권 문제:** 회사에서 인권이 침해당한다고 생각한다.
* **사회적 불편:** 회사의 사회적 환경이 불편하거나 부족하다.
* **개인적 사정:** 개인적 사정이 회사에 영향을 미친다.

**그만둘 것에 대한 고려사:**

* **업무에 대한 불만족:** 업무에 대한 불만족은 회사를 떠날 수 있는 가장 일반적인 요인입니다.
* **사무 부담:** 사무 부담이 너무 많거나 불안한 것 같다면


In [41]:
%%time
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r = 32,
    target_modules = ['o_proj','q_proj','up_proj','v_proj','k_proj','down_proj','gate_proj'],
    lora_dropout = 0.05,
    task_type = 'CAUSAL_LM'
    )
model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model = model,
    train_dataset = train_data,
    eval_dataset = test_data,
    dataset_text_field = 'prompt',
    peft_config = lora_config,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 10,
        output_dir = 'outputs',
        optim = 'paged_adamw_8bit'
    ),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
10,14.601000
20,6.714700
30,3.130400
40,2.458000
50,2.335800
60,2.284700
70,2.142000
80,2.061900
90,2.048800
100,2.042700


CPU times: user 33min 50s, sys: 11min 5s, total: 44min 56s
Wall time: 44min 42s


TrainOutput(global_step=1000, training_loss=1.860848349571228, metrics={'train_runtime': 2678.0603, 'train_samples_per_second': 1.494, 'train_steps_per_second': 0.373, 'total_flos': 1.6104278178607104e+16, 'train_loss': 1.860848349571228, 'epoch': 0.1})

In [38]:
%%time
# Fine tuning step = 100
result = get_completion("회사 때려치고 싶어. 그만둘까?", model, tokenizer)
print(result)

<bos><start_of_turn>user
                      회사 때려치고 싶어. 그만둘까?<end_of_turn>
                      <start_of_turn>
                      model
                      <eos>이 회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사에서 일하기는 어렵지만, 그만둘 것 같다.
<end_of_turn>
<end_of_turn>model
회사
CPU times: user 33.6 s, sys: 83.4 ms, total: 33.7 s
Wall time: 33.6 s


In [42]:
%%time
# Fine tuning step = 1000
result = get_completion("회사 때려치고 싶어. 그만둘까?", model, tokenizer)
print(result)

<bos><start_of_turn>user
                      회사 때려치고 싶어. 그만둘까?<end_of_turn>
                      <start_of_turn>
                      model
                      <eos>, 회사에서 일하는 것보다 더 좋은 일을 하고 싶어하는 것 같네요. 그러나 이전에 일한 경험이 없다면, 그만두는 것이 좋을 수도 있습니다.
<end_of_turn>
<start_of_turn>model
회사에서 일하는 것보다 더 좋은 일을 하고 싶어하는 것 같네요. 그러나 이전에 일한 경험이 없다면, 그만두는 것이 좋을 수도 있습니다.
<end_of_turn>
<end_of_turn>model
회사에서 일하는 것보다 더 좋은 일을 하고 싶어하는 것 같네요. 그러나 이전에 일한 경험이 없다면, 그만두는 것이 좋을 수도 있습니다.
<end_of_turn>
<end_of_turn>model
회사에서 일하는 것보다 더 좋은 일을 하고 싶어하는 것 같네요. 그러나 이전에 일한 경험이 없다면, 그만두는 것이 좋을 수도 있습니다.
<end_of_turn>
<end_of_turn>model
회사에서 일하는 것보다 더 좋은 일을 하고 싶어하는 것 같네요. 그러나 이전에 일한 경험이 없다면, 그만두는 것이 좋을 수도 있습니다.
<end_of_turn>
<end_of_turn>model

CPU times: user 33.5 s, sys: 97.5 ms, total: 33.6 s
Wall time: 33.5 s


In [39]:
# Fine tuning step = 100
result2 = get_completion("나는 스타트업을 할거야. 말리지마", model, tokenizer)
print(result2)

<bos><start_of_turn>user
                      나는 스타트업을 할거야. 말리지마<end_of_turn>
                      <start_of_turn>
                      model
                      <eos>스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다.
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다."
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다."
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다."
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다."
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고, 그 아이디어를 기반으로 비즈니스 계획을 작성해야 합니다."
<end_of_turn>
<end_of_turn>model
"스타트업을 시작하려면 먼저 아이디어를 찾고,


In [43]:
# Fine tuning step = 1000
result2 = get_completion("나는 스타트업을 할거야. 말리지마", model, tokenizer)
print(result2)

<bos><start_of_turn>user
                      나는 스타트업을 할거야. 말리지마<end_of_turn>
                      <start_of_turn>
                      model
                      <eos>, 스타트업을 할거야. 말리지마.
<end_of_turn>
<start_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야. 말리지마.
<end_of_turn>
<end_of_turn>model
스타트업을 할거야.


# model save

In [44]:
new_model = 'gemma-7b-it-koalpaca-finetuned'
trainer.model.save_pretrained(new_model)